### 선행 작업
1. wget으로 데이터 압축파일(.gz) 다운로드
2. 모든 압축파일 압출 풀기

In [2]:
import os
import jsonlines
import json
import re
import string
import nltk
from nltk.data import load

In [50]:
nltk.download('punkt')

tokenizer = load('tokenizers/punkt/english.pickle')   # nltk.sent_tokenizer가 불러오는 파일
extra_abbreviation = [    
    'RE', 're', 'pat', 'no', 'nos', 'vol', 'jan', 'feb', 'mar', 'apr', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec',
    'eng', 'ser', 'ind', 'ed', 'pp', 'e.g', 'al', 'T.E.N.S', 'E.M.S', 'F.E', 'U.H.T.S.T', 'degree', '/g', 'm',
    'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z'
]
tokenizer._params.abbrev_types.update(extra_abbreviation)   # _params로 파라미터에 직접 접근, 업데이트

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lynn1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#### 지정 필요
- start_file(파일 시작 번호)
- end_file(파일 마지막 번호)
- data_dir(데이터 폴더 위치)
- sent_file(최종 txt 파일 이름, 위치)

In [102]:
start_file = 1000   # s2-corpus-000 등 파일 번호
end_file = 1999
data_dir = 'C:/Users/lynn1/OneDrive/바탕 화면/Capstone/data/open_research'

# for i in range(start_file, end_file+1):
#     old_name = data_dir + '/s2-corpus-00{}'.format(i)
#     new_name = old_name + '.json'
#     os.rename(old_name, new_name)

In [ ]:
whole_lst=[]
nums=[]        # 작업한 파일 번호 저장

In [103]:
# paperAbstract 데이터만 추출
print('appending...')
for i in range(start_file, end_file+1):
    if i<10:
        idx = '00{}'.format(i)
    elif i<100:
        idx = '0{}'.format(i)
    else:
        idx = '{}'.format(i)
        
    with jsonlines.open(data_dir+'/s2-corpus-{}'.format(idx), 'r') as file:
        data_lst = [line['paperAbstract'] for line in file if line['paperAbstract']!='']
        whole_lst = whole_lst + data_lst
    nums.append(i)
    print('>'+str(i), end='')
print('\nlines:', len(whole_lst))

1200 > 19612|1201 > 39250|1202 > 58713|1203 > 78432|1204 > 98148|1205 > 117748|1206 > 137345|1207 > 156810|1208 > 176307|1209 > 195800|1210 > 215387|1211 > 234842|1212 > 254337|1213 > 273939|1214 > 293623|1215 > 313172|1216 > 332764|1217 > 352346|1218 > 371776|1219 > 391456|1220 > 410991|1221 > 430583|1222 > 450221|1223 > 469870|1224 > 489568|1225 > 509046|1226 > 528725|1227 > 548478|1228 > 568104|1229 > 587635|1230 > 607234|1231 > 626908|1232 > 646514|1233 > 666207|1234 > 685651|1235 > 705333|1236 > 724985|1237 > 744493|1238 > 763990|1239 > 783722|1240 > 803349|1241 > 822933|1242 > 842555|1243 > 862107|1244 > 881575|1245 > 901206|1246 > 920690|1247 > 940244|1248 > 959759|1249 > 979351|1250 > 998932|1251 > 1018326|1252 > 1038016|1253 > 1057538|1254 > 1077073|1255 > 1096707|1256 > 1116270|1257 > 1135900|1258 > 1155675|1259 > 1175177|1260 > 1194839|1261 > 1214356|1262 > 1233893|1263 > 1253349|1264 > 1272906|1265 > 1292440|1266 > 1312080|1267 > 1331512|1268 > 1351076|1269 > 1370557|1270 >

In [104]:
# 모든 코퍼스 파일의 paperAbstract 문장으로 분리하여 txt 파일에 저장(계속 이어씀)
# whole_lst에 모든 데이터 저장 후 한번에 생성하기
sent_file = open(data_dir+'/separate_sents(1000~1999).txt', 'a', encoding='utf-8')
no_blank = False
print('writing...')
for i, line in enumerate(whole_lst):
    if i%10000==0:
        print('>'+str(i), end='')

    if line=='':
        break
    if line.strip()=='':
        if no_blank:
            continue
        sent_file.write(f'{line}')
    else:
        result_ = nltk.sent_tokenize(line)   # 문장 구조 학습한 모델 -> 약어에 쓰이는 마침표 등 학습됨
        result = [f'{cul_line}\n' for cul_line in result_]
        for save_line in result:
            sent_file.write(save_line)
sent_file.close()